In [1]:
# === One-click run + Sweep ===
import os, json, yaml
from normwear_utill import run_experiment, run_sweep

CONFIG_PATH = r"D:\Labroom\SDPhysiology\base.yaml"  # 공통 기본값
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    base_cfg = yaml.safe_load(f)

# 공통: split seed 고정(공정 비교)
print("== Base config loaded ==")
print(json.dumps(base_cfg, ensure_ascii=False, indent=2))

# ----- 단일 러닝(현재 최선 가정값) -----
single_overrides = {
  # label lag/stride/pooling/PCA만 바꿔도 됩니다
  "label_lag_sec": -2,
  "windowing": {"stride_seconds": 2},
  "embed": {"pooling": "meanstd_concat", "pca_dim": 256, "pca_whiten": False},
}
out = run_experiment(base_cfg, overrides=single_overrides)

# ----- 스윕 예시(라벨 래깅/stride/풀링/PCA 조합) -----
sweep = []
for lag in [-3, -2, -1, 0]:
    sweep.append({"label_lag_sec": lag})
# (원하면 아래도 추가해서 다변량 스윕)
# for lag in [-2, -1]:
#   for stri in [1, 2]:
#     for pool in ["meanstd_concat", "concat"]:
#       for pca in [128, 256]:
#         sweep.append({
#           "label_lag_sec": lag,
#           "windowing": {"stride_seconds": stri},
#           "embed": {"pooling": pool, "pca_dim": pca}
#         })

outs = run_sweep(base_cfg, sweep)
print("Top-level outputs saved. (각 런 폴더의 results.json과 sweep_summary.csv를 확인)")


== Base config loaded ==
{
  "data": {
    "data_dir": "D:\\Labroom\\SDPhysiology\\Data\\processed_individual_anonymized",
    "output_dir": "D:\\Labroom\\SDPhysiology\\runs",
    "scenes": [
      "Hallway"
    ]
  },
  "windowing": {
    "fs_raw": 120,
    "fs_target": 50,
    "window_seconds": 30,
    "stride_seconds": 2
  },
  "channels": {
    "channel_order": [
      "PPG_Clean",
      "EDA_Clean",
      "RSP_Clean"
    ]
  },
  "normalization": {
    "waveform_norm": "zscore10s",
    "label_mode": "zscore10s",
    "baseline_seconds": 10
  },
  "quality": {
    "enable_ppg_quality_filter": true,
    "ppg_quality_threshold": 0.7
  },
  "scale": {
    "max_windows_per_pid": 24
  },
  "split": {
    "train_pids": [],
    "test_pids": [],
    "train_count": 12,
    "test_count": 4,
    "seed": 42
  },
  "embed": {
    "normwear_repo": "C:\\Users\\user\\code\\NormWear",
    "weight_path": "C:\\Users\\user\\code\\SDPhysiology\\weights\\normwear\\normwear_last_checkpoint-15470-correct.p

c:\Users\user\anaconda3\envs\normwear\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\normwear\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\user\code\NormWear\main_model.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only

Model Checkpoint is successfully loaded!
 - NormWearModel 로드 완료
 - 임베딩 완료: N=288, C=3, T=1500, Z.shape=(288, 4608), 소요=130.05s, pooling=meanstd_concat, batch=16
 - 임베딩 완료: N=96, C=3, T=1500, Z.shape=(96, 4608), 소요=43.38s, pooling=meanstd_concat, batch=16
 - PCA: 4608 → 256 | explained_var=1.000
 - Saved embeddings to cache: D:\Labroom\SDPhysiology\runs\_emb_cache_global\c2a5ff4015
 - 임베딩 캐시 저장 완료: D:\Labroom\SDPhysiology\runs\Hallway__lag-2__str2__pool-meanstd_concat__pca256\_emb_cache
 - Local cache linked: D:\Labroom\SDPhysiology\runs\Hallway__lag-2__str2__pool-meanstd_concat__pca256\_emb_cache
 - [baseline] R2=-0.4008, MAE=0.6546, RMSE=0.9764
[MODEL] svr
[CV] model=svr | scoring=neg_mean_absolute_error | splits=5 | scale_y=True
 - best_params: {'regressor__svr__C': 1, 'regressor__svr__epsilon': 0.3, 'regressor__svr__gamma': 0.0001}
 - cv best neg_mean_absolute_error = -1.660029
 - [TEST] R2=-0.0210, MAE=0.6068, RMSE=0.8336
[MODEL] ridge
[CV] model=ridge | scoring=neg_mean_absolute_e